# Multi machine hyperopt
Here are the steps for the hyperopt flow
- Define a function to minimize using spark MLlib
- Define a search space over hyper parameters
- Select the search algorithm
- Run the tuning algorithm with Hyperopt

We will be using hyperopt and sparks mllib to perform a distributed tuning of hyperparameters while training a ml algorithm


In [0]:
display(dbutils.fs.ls('/databricks-datasets'))

path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,1727855155408
dbfs:/databricks-datasets/README.md,README.md,976,1532502332000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,1727855155408
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1455505270000
dbfs:/databricks-datasets/adult/,adult/,0,1727855155408
dbfs:/databricks-datasets/airlines/,airlines/,0,1727855155408
dbfs:/databricks-datasets/amazon/,amazon/,0,1727855155408
dbfs:/databricks-datasets/asa/,asa/,0,1727855155408
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,1727855155408
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,1727855155408


In [0]:
display(dbutils.fs.ls('/databricks-datasets/mnist-digits'))

path,name,size,modificationTime
dbfs:/databricks-datasets/mnist-digits/README.md,README.md,640,1455505289000
dbfs:/databricks-datasets/mnist-digits/data-001/,data-001/,0,1727855166376


In [0]:
display(dbutils.fs.ls('/databricks-datasets/mnist-digits/data-001'))

path,name,size,modificationTime
dbfs:/databricks-datasets/mnist-digits/data-001/mnist-digits-test.txt,mnist-digits-test.txt,11671108,1455505289000
dbfs:/databricks-datasets/mnist-digits/data-001/mnist-digits-train.txt,mnist-digits-train.txt,69430283,1455505289000


In [0]:
# Read the README.md file
readme_path = 'dbfs:/databricks-datasets/mnist-digits/README.md'
readme_content = dbutils.fs.head(readme_path)

# Display the content as HTML
displayHTML(f"<pre>{readme_content}</pre>")

======================================
MNIST handwritten digits dataset
======================================

## Data Source
LibSVM Datasets
https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass.html#mnist
Chih-Chung Chang and Chih-Jen Lin, LIBSVM : a library for support vector machines. ACM Transactions on Intelligent Systems and Technology, 2:27:1--27:27, 2011.

## Original Data Set Source
Yann LeCun, L. Bottou, Y. Bengio, and P. Haffner. 
Gradient-based learning applied to document recognition. 
Proceedings of the IEEE, 86(11):2278-2324, November 1998. 
MNIST database available at http://yann.lecun.com/exdb/mnist/

In [0]:
# Run distributed training using MLlib
full_training_data = spark.read.format("libsvm").load('dbfs:/databricks-datasets/mnist-digits/data-001/mnist-digits-train.txt')
test_data = spark.read.format("libsvm").load('dbfs:/databricks-datasets/mnist-digits/data-001/mnist-digits-train.txt')

full_training_data.cache()
test_data.cache()

print(f"There are {full_training_data.count()} training images and {test_data.count()} test images")

There are 60000 training images and 60000 test images


In [0]:
# Lets random split the training_data for tuning
training_data, validation_data = full_training_data.randomSplit([0.8, 0.2], seed=42)

In [0]:
import mlflow
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


In [0]:
try:
    import mlflow.pyspark.ml
    mlflow.pyspark.ml.autolog()
except:
    print(f"Your version of mlflow is {mlflow.__verion__}, try upgrading to use auto logging")

In [0]:
# Lets train a decision tree classifier
def train_tree(minInstancesPerNode, maxBins):
    with mlflow.start_run(nested=True):
        indexer = StringIndexer(inputCol="label", outputCol="indexedLabel")
        dtc = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="features", minInstancesPerNode=minInstancesPerNode, maxBins=maxBins)
        # Create the pipe
        pipeline = Pipeline(stages=[indexer, dtc])
        # run the pipe
        model = pipeline.fit(training_data)

        # define evaluation metric
        evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
        predictions = model.transform(test_data)
        validation_metric = evaluator.evaluate(predictions)
        mlflow.log_metric("val_f1_score", validation_metric)

        return model,validation_metric

In [0]:
init_model, valid_metric = train_tree(minInstancesPerNode=20, maxBins=2)
print("Trained decision tree achieved and F1 score of", valid_metric)

2024/10/02 07:46:29 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.
2024/10/02 07:46:45 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('features', VectorUDT(), True)]. Model signature is not logged.
2024/10/02 07:47:06 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2024/10/02 07:47:59 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-07-14; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'databricks-feature-engineering'}
2024/10/02 07:47:59 WARNING mlflow.u

Trained decision tree achieved and F1 score of 0.6722346573614189


# Lets do hyperopt to get best models


In [0]:
from hyperopt import fmin, tpe, hp, STATUS_OK
def train_tree_hyperopt(params):
    minInstancesPerNode = int(params['minInstancesPerNode'])
    maxBins = int(params['maxBins'])
    model, validation_metric = train_tree(minInstancesPerNode, maxBins)
    loss = -validation_metric
    return {'loss': loss, 'status': STATUS_OK}


In [0]:
import numpy as np
space = {
    'minInstancesPerNode': hp.uniform('minInstancesPerNode',2,5),
    'maxBins': hp.uniform('maxBins',2,5)

}

In [0]:
# Running the tuning algorithm with hyperopt. We do not need to pass trials to fmin(), by default it will use the 'Trials' class to run the trials on driver nodes.
# Donot use SparkTrials with MlLib class, the class is desgined to work with algorithms that are not them selves distributed. MLlib uses distributed computing already and is not compatible with spark Trials.

In [0]:
with mlflow.start_run(): # track progress with mlflow.start_run()
    best_params = fmin(fn=train_tree_hyperopt, space=space, algo = tpe.suggest, max_evals=10)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/10/02 07:48:13 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.

2024/10/02 07:48:39 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().

2024/10/02 07:49:29 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-07-14; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'databricks-feature-engineering'}

2024/10/02 07:49:30 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for 

 10%|█         | 1/10 [01:25<12:45, 85.06s/trial, best loss: -0.6802141350126889]

2024/10/02 07:49:38 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.

2024/10/02 07:50:03 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().

2024/10/02 07:50:53 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-07-14; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'databricks-feature-engineering'}

2024/10/02 07:50:55 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for 

 20%|██        | 2/10 [02:50<11:19, 84.99s/trial, best loss: -0.6808065313193914]

2024/10/02 07:51:03 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.

2024/10/02 07:51:28 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().

2024/10/02 07:52:17 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-07-14; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'databricks-feature-engineering'}

2024/10/02 07:52:19 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for 

 30%|███       | 3/10 [04:13<09:50, 84.38s/trial, best loss: -0.6808065313193914]

2024/10/02 07:52:27 WARNING mlflow.data.spark_dataset: Failed to infer schema for Spark dataset. Exception: Unsupported Spark Type '<class 'pyspark.ml.linalg.VectorUDT'>', MLflow schema is only supported for scalar Spark types.

2024/10/02 07:52:53 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().



In [0]:
print("Best value found for parameters:",best_params)

# Retrain the model on training dataset using best parameter values
### Run was stopped as it was getting expensive

In [0]:
best_minInstancesPerNode = 36
best_maxBins = 25

final_model ,val_f1_score = train_tree(best_minInstancesPerNode,best_maxBins)

In [0]:
print("Validation F1 score:",val_f1_score)

In [0]:
# Lets test the dataset to compare evaluation metrics for initial and best model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
init_model_test_metric = evaluator.evaluate(init_model.transform(test_data))
final_model_test_metric = evaluator.evaluate(final_model.transform(test_data))

In [0]:
print(f"Test data metrics untuned model f1 score:{init_model_test_metric} vs tuned model f1 score: {final_model_test_metric}")